# Lets Create Some Random Graphs

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages')
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import random
import math

In [ ]:
def generarGrafoPesosNormales(f, c, mu=5, sigma=1, p_arista=0.5):
    assert(f<=c)

    grafo = []
    verticesUsados = []
    aristasUsadas = []

    # Genera un grafo minimal con respecto a que cualquier cliente
    # sea alcanzable desde al menos una fábrica.

    for cliente in range(f+1, f+c+1):
        # Se asegura de que cada cliente sea adyacente a una fábrica
        # o a otro cliente tal que hay un camino entre él y una fábrica.
        vecino = np.random.choice([i for i in range(1,f+1)] + verticesUsados)
        verticesUsados.append(cliente)
        l, r = min(cliente, vecino), max(cliente, vecino)
        grafo.append([ l, r, int(max(np.random.normal(mu, sigma), 0)) ])
        aristasUsadas.append((l,r))

    # Agrega más aristas con probabilidad p_arista.
    # Cada una es independiente.

    for i in range(1, f+c+1):
        for j in range(i+1, f+c+1):
            if not((i,j) in aristasUsadas):
                p = np.random.uniform()
                if p <= p_arista:
                    grafo.append([ i, j, int(max(np.random.normal(mu, sigma), 0)) ])

    grafoStr = [ " ".join( str(k) for k in e ) for e in grafo ]

    return grafoStr

def generarEntradaPropClientes(n, mu=5, sigma=1, p_arista=0.5, archivo="output"):
    # Genera un caso de test con 32 grafos de n nodos. Los pesos están distribuidos
    # normalmente según los parámetros especificados.
    minimo = n // 2 if n%2 == 0 else n // 2 + 1

    with open(archivo, 'w') as entrada:
        for k in range(minimo,n):    
            for i in range(50):
                grafo = generarGrafoPesosNormales(n-k, k, mu, sigma, p_arista)
                entrada.write(str(n-k) + " " + str(k) + " " + str(len(grafo)) + '\n')
                for j in grafo:
                    entrada.write(j + '\n')

        entrada.write("0\n")
        
def generarEntradaCantidadesFijas(f, c, mu=5, sigma=1, archivo="output"):
    # Genera un caso de test con 32 grafos de n nodos. Los pesos están distribuidos
    # normalmente según los parámetros especificados.
    with open(archivo, 'w') as entrada:
        for p in np.arange(0.0, 1.1, 0.05):
            for i in range(80):
                grafo = generarGrafoPesosNormales(f, c, mu, sigma, p)
                entrada.write(str(f) + " " + str(c) + " " + str(len(grafo)) + '\n')
                for j in grafo:
                    entrada.write(j + '\n')
        entrada.write("0\n")

In [ ]:
for p in np.arange(0.0, 1.1, 0.25):
    generarEntradaPropClientes(50, 20, 5, p, "tests/proporcionClientes_{0}.test-in".format(p))

In [ ]:
for j in range(10, 40, 10):
    generarEntradaCantidadesFijas(j, j+10, 20, 5, "tests/densidadCreciente_{0}_{1}.test-in".format(j, j+10))

In [ ]:
for p in np.arange(0.0, 1.1, 0.25):
    generarEntradaPropClientes(50, 20, 5, p, "tests/ccreciente_{0}.test-in".format(p))

# Corremos el experimento

In [ ]:
! make expe

In [ ]:
results = {}

for p in np.arange(0.0, 1.1, 0.25):
    results["proporcionClientes_{0}".format(p)] = pd.read_csv("./experimentacion/proporcionClientes_{0}.stderr".format(p), delimiter=";")


In [ ]:
results["proporcionClientes_0.25"].tail()

In [ ]:
df = pd.DataFrame({'Arboles': results["proporcionClientes_0.0"]['Tamaño de la solucion'],
                  'P = 0.25': results["proporcionClientes_0.25"]['Tamaño de la solucion'],
                  'P = 0.5': results["proporcionClientes_0.5"]['Tamaño de la solucion'],
                  'P = 0.75': results["proporcionClientes_0.75"]['Tamaño de la solucion'],
                  'Completos': results["proporcionClientes_1.0"]['Tamaño de la solucion']})

ax = df.plot(logy=False)
ax.set_ylabel(u"Cantidad de aristas")
ax.set_xlabel(u"Cantidad de clientes")

plt.title(u"Cantidad de aristas de la solución para varias densidades del grafo")
plt.show()

## Experimento 6

In [ ]:
resultsD = {}
    
for j in range(10, 40, 10):
    resultsD["densidadCreciente_{0}_{1}".format(j, j+10)] = pd.read_csv("./experimentacion/densidadCreciente_{0}_{1}.stderr".format(j, j+10), delimiter=";")


In [ ]:
resultsD["densidadCreciente_30_40"].tail()

In [ ]:
for key, value in resultsD.items():
    resultsD[key] = value.groupby("Cantidad de rutas").mean()

resultsD["densidadCreciente_30_40"]

In [ ]:
df = pd.DataFrame({'30 nodos': resultsD["densidadCreciente_10_20"]['Tiempo de ejecucion'],
                  '50 nodos': resultsD["densidadCreciente_20_30"]['Tiempo de ejecucion'],
                  '70 nodos': resultsD["densidadCreciente_30_40"]['Tiempo de ejecucion']})

ax = df.plot(logy=True)
ax.set_ylabel(u"Tiempo de ejecución", fontsize=14)
ax.set_xlabel(u"Cantidad de aristas", fontsize=14)

plt.title(u"Tiempo de ejecución según la densidad del grafo", fontsize=14)
plt.show()

### C creciente

In [ ]:
resultsC = {}
for p in np.arange(0.0, 1.1, 0.25):
    resultsC["ccreciente_{0}".format(p)] = pd.read_csv("experimentacion/ccreciente_{0}.stderr".format(p), delimiter=";")

In [ ]:
resultsC["ccreciente_0.0"]

In [ ]:
for key, value in resultsC.items():
    resultsC[key] = value.groupby("Cantidad de clientes").mean()
    
resultsC["ccreciente_0.0"]

In [ ]:
df = pd.DataFrame({'Arboles': resultsC["ccreciente_0.0"]['Tiempo de ejecucion'],
                  'P = 0.25': resultsC["ccreciente_0.25"]['Tiempo de ejecucion'],
                  'P = 0.5': resultsC["ccreciente_0.5"]['Tiempo de ejecucion'],
                  'P = 0.75': resultsC["ccreciente_0.75"]['Tiempo de ejecucion'],
                  'Completos': resultsC["ccreciente_1.0"]['Tiempo de ejecucion']})

ax = df.plot(logy=False)
ax.set_ylabel(u"Tiempo de ejecución en segundos")
ax.set_xlabel(u"Cantidad de clientes")

plt.title(u"Tiempo de ejecución según la cantidad de clientes")
plt.show()

## Contraste con la complejidad teórica

Necesitamos un Pearson que muestre la correlación entre la complejidad empírica y O(R log(C)). 

In [ ]:
sns.jointplot(dfComp['Complejidad n**2'], dfComp['Tiempo en segundos de completos'], kind="reg")
sns.jointplot(dfComp['Complejidad n'], dfComp['Tiempo en segundos de arboles'], kind="reg")


plt.show()

In [ ]:
dfRoot = pd.DataFrame({'Arboles': results["dfNormal_0.0"]['Tiempo calculando raiz'],
                  'P = 0.25': results["dfNormal_0.25"]['Tiempo calculando raiz'],
                  'P = 0.5': results["dfNormal_0.5"]['Tiempo calculando raiz'],
                  'P = 0.75': results["dfNormal_0.75"]['Tiempo calculando raiz'],
                  'Completos': results["dfNormal_1.0"]['Tiempo calculando raiz']})

ax = dfRoot.plot(logy=False)
ax.set_ylabel("Tiempos de ejecución en Segundos")
ax.set_xlabel("Cantidad de nodos")

plt.title("Tiempos para encontrar el MST")
plt.show()

In [ ]:
dfComp = pd.DataFrame()
dfComp['Cantidad de Nodos'] = results["dfNormal_1.0"].index.values
dfComp['Complejidad n**2'] = [100 + random.normalvariate(n**2,2) for n in results["dfNormal_0.0"].index.values]
dfComp['Complejidad n'] = [100 + random.normalvariate(n,0.3) for n in results["dfNormal_0.0"].index.values]

dfComp['Tiempo de encontrar raiz en segundos de ex-completos'] =  results["dfNormal_1.0"]['Tiempo calculando raiz']
dfComp['Tiempo de encontrar raiz en segundos de arboles'] =  results["dfNormal_0.0"]['Tiempo calculando raiz']
plt.clf()
correlation = dfComp.corr()
correlation

In [ ]:
sns.jointplot(dfComp['Complejidad n'], dfComp['Tiempo de encontrar raiz en segundos de ex-completos'], kind="reg")
sns.jointplot(dfComp['Complejidad n'], dfComp['Tiempo de encontrar raiz en segundos de arboles'], kind="reg")

plt.show()

# Super Test

In [ ]:
with open("tests/rlogc.test-in", 'w') as entrada:
    for c in range(5,21):
        for p in np.arange(0.0, 1.1, 0.01):
            for i in range(80):
                grafo = generarGrafoPesosNormales(5, c, 20, 5, p)
                entrada.write("5 " + str(c) + " " + str(len(grafo)) + '\n')
                for j in grafo:
                    entrada.write(j + '\n')
    entrada.write("0\n")

In [ ]:
!make expe

In [ ]:
results = {}

dfRlogc = pd.read_csv("./experimentacion/rlogc.stderr", delimiter=";")

In [ ]:
dfRlogc = dfRlogc.groupby(["Cantidad de rutas", "Cantidad de clientes"]).mean()
dfRlogc

In [ ]:
dfComp = pd.DataFrame()
dfComp['Complejidad rlogc'] = [x[0]*math.log(x[1]) for x in dfRlogc.index.get_values()]
dfComp['Tiempo de ejecucion'] =  [x for x in dfRlogc['Tiempo de ejecucion']]
plt.clf()
correlation = dfComp.corr()
correlation

In [ ]:
sns.jointplot(dfComp['Complejidad rlogc'], dfComp['Tiempo de ejecucion'], kind="reg")

plt.show()